<a href="https://colab.research.google.com/github/Freaker99/car-detection/blob/main/car_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
from google.colab import drive
import os

drive.flush_and_unmount()
drive.mount('/content/drive')
root = '/content/drive/My Drive/car-detection/'

Mounted at /content/drive


In [ ]:
#!pip install opencv-python
#!pip install torch torchvision
#!pip install numpy
#!pip install ultralytics

In [ ]:
#!git clone https://github.com/ultralytics/yolov5
#%cd yolov5
#!pip install -r requirements.txt
#!wget https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5s.pt

In [ ]:
!pip install yolov5
!pip install --upgrade yolov5

# Eksperymenty z DataLoader

In [ ]:
# Importowanie niezbędnych bibliotek
import os
import torch
import json

from PIL import Image
from torchvision.datasets import VisionDataset
from torchvision.transforms import transforms
from torchvision.transforms.functional import resize

# Klasa datasetu BDD100K
class BDD100KDataset(VisionDataset):
    def __init__(self, labels_file, image_dir, transform=None, target_transform=None, resize_size=(256, 256), crop_size=(224, 224)):
        super(BDD100KDataset, self).__init__(image_dir, transform=transform, target_transform=target_transform)
        self.labels = self._read_labels(labels_file)
        self.images = [item["name"] for item in self.labels]
        self.resize_size = resize_size
        self.crop_size = crop_size

    def __getitem__(self, index):
        image_path = os.path.join(self.root, self.images[index])
        image = Image.open(image_path).convert("RGB")
        image = transforms.Resize(self.resize_size)(image)  # Przekształcenie zmiany rozmiaru
        image = transforms.RandomCrop(self.crop_size)(image)  # Przekształcenie losowego wycięcia
        label = self.labels[index]
        if self.transform is not None:
            image = self.transform(image)
        if self.target_transform is not None:
            label = self.target_transform(label)
        return image, label

    def __len__(self):
        return len(self.labels)

    def _read_labels(self, labels_file):
        with open(labels_file, "r") as f:
            labels = json.load(f)
        return labels

In [ ]:
def transform(image, target):
    image = image.resize((224, 224))
    image = torch.Tensor(np.array(image)).permute(2, 0, 1) / 255.0  # Konwersja na tensor i normalizacja
    target = torch.Tensor(target)
    return image, target

In [ ]:
from numpy import True_
# Ścieżki do folderów z obrazami
image_dir_train = '/content/drive/My Drive/car-detection/images/train/train100k'
image_dir_val = '/content/drive/My Drive/car-detection/images/val/'

# Ścieżki do plików etykiet
train_labels_file = '/content/drive/My Drive/car-detection/labels/det_train.json'
val_labels_file = '/content/drive/My Drive/car-detection/labels/det_val.json'

# Tworzenie instancji datasetu treningowego i walidacyjnego
train_dataset = BDD100KDataset(train_labels_file, image_dir_train, transform=transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop((224, 224)),
    transforms.ToTensor(),
]))
val_dataset = BDD100KDataset(val_labels_file, image_dir_val, transform=transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop((224, 224)),
    transforms.ToTensor(),
]))

# Definiowanie DataLoader dla treningu i walidacji
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=0, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=0)

In [ ]:
import torch
import torch.optim as optim
from torchvision.models import resnet50
from torch.utils.data import DataLoader
from yolov5.models.yolo import Model
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pin_memory = True

# Funkcja treningowa
def train(model, train_loader, criterion, optimizer):
    model.train()
    total_loss = 0.0

    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    return avg_loss

In [ ]:
# Inicjalizacja modelu YOLOv5
model = Model(root + "yolov5/models/yolov5s.yaml")

# Załadowanie wstępnie wytrenowanych wag modelu
state_dict = torch.load(root + "yolov5/yolov5s.pt")
model.load_state_dict(state_dict, strict=False)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = torch.nn.CrossEntropyLoss()

# Ustawienie trybu trenowania modelu
model.train()

# Definicja optymalizatora
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Definicja funkcji straty
criterion = torch.nn.CrossEntropyLoss()

# Parametry trenowania
num_epochs = 10
batch_size = 2
learning_rate = 0.001



# Pętla trenowania
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # Trening
    train_loss = train(model, train_loader, criterion, optimizer)
    print(f"Train Loss: {train_loss:.4f}")

    # Ocena na zbiorze walidacyjnym (opcjonalnie)
    # val_loss = evaluate(model, val_loader, criterion)
    # print(f"Val Loss: {val_loss:.4f}")

    print("")

# Zapisanie wytrenowanego modelu
torch.save(model.state_dict(), "trained_model.pt")

**Tools**

In [ ]:
import os
import json

def split_large_file(input_file, output_folder):
    with open(input_file, 'r') as f:
        data = json.load(f)

    for label in data:
        name = label['name'].replace('.jpg', '.txt')
        output_path = os.path.join(output_folder, name)

        with open(output_path, 'w') as txt_file:
            json.dump(label, txt_file)

        print(f"Zapisano plik: {name}")

# Wprowadź ścieżkę do swojego dużego pliku tekstowego
input_file = '/content/drive/My Drive/car-detection/labels/det_val_txt.txt'

# Określ folder, w którym będą zapisywane podzielone pliki
output_folder = '/content/drive/My Drive/car-detection/labels/labels_val_txt'

# Wywołaj funkcję do podziału pliku
split_large_file(input_file, output_folder)

In [28]:
import os
import json

def convert_to_yolov5(input_folder, output_folder):
    # Słownik mapujący klasy na indeksy w formacie YOLOv5
    class_mapping = {
        "pedestrian": 0,
        "bicycle": 1,
        "car": 2,
        "motorcycle": 3,
        "bus": 5,
        "train": 6,
        "truck": 7,
        "traffic light": 9,
        "traffic sign": 11
        # Brak klasy "rider", bo jest nieistotna
    }

    # Sprawdź istnienie folderu wyjściowego; jeśli nie istnieje, utwórz go
    os.makedirs(output_folder, exist_ok=True)

   # Przetwarzaj każdy plik w folderze wejściowym
    for filename in os.listdir(input_folder):
        input_file = os.path.join(input_folder, filename)
        output_file = os.path.join(output_folder, filename)

        with open(input_file, 'r') as f:
            data = json.load(f)

        with open(output_file, 'w') as f:
            for label in data['labels']:
                category = label['category']
                if category in class_mapping:
                    class_index = class_mapping[category]

                    x1 = label['box2d']['x1']
                    y1 = label['box2d']['y1']
                    x2 = label['box2d']['x2']
                    y2 = label['box2d']['y2']

                    # Oblicz współrzędne środka oraz szerokość i wysokość bounding boxa
                    width = x2 - x1
                    height = y2 - y1
                    x_center = x1 + width / 2
                    y_center = y1 + height / 2

                    # Przeskaluj współrzędne do zakresu 0-1
                    x_center /= 1280
                    y_center /= 720
                    width /= 1280
                    height /= 720

                    # Zapisz wynikowy wiersz w formacie YOLOv5
                    f.write(f"{class_index} {x_center} {y_center} {width} {height}\n")

        print(f"Przetworzono plik: {filename}")

# Wprowadź ścieżki do swojego folderu wejściowego i docelowego folderu YOLOv5
input_folder = "/content/drive/My Drive/car-detection/testowanie/l1"
output_folder = "/content/drive/My Drive/car-detection/testowanie/l2"

convert_to_yolov5(input_folder, output_folder)

Przetworzono plik: Kopia bba9d58b-18c62a06.txt
Przetworzono plik: Kopia bab48a4e-43fa0dfe.txt
Przetworzono plik: Kopia b6219281-9780cb21.txt
Przetworzono plik: Kopia b8697036-9589cddf.txt
Przetworzono plik: Kopia b8685330-d275b328.txt


testowanie działa, teraz wszystkie

In [ ]:
import json
import os

def check_json_syntax(folder_path):
    file_names = os.listdir(folder_path)

    for file_name in file_names:
        file_path = os.path.join(folder_path, file_name)

        with open(file_path, 'r') as f:
            try:
                json.load(f)
                print(f"Plik {file_name} ma poprawną składnię JSON.")
            except json.JSONDecodeError as e:
                print(f"Błąd składni JSON w pliku {file_name}: {e}")

# Wprowadź ścieżkę do folderu zawierającego pliki tekstowe
folder_path = "/content/drive/My Drive/car-detection/labels/labels_val_txt"

# Wywołaj funkcję sprawdzającą składnię JSON
check_json_syntax(folder_path)

In [19]:
# Wyświetl zawartość pliku

# dobry
#file_path = "/content/drive/My Drive/car-detection/labels/labels_val_txt/c79e169f-054f74e7.txt"

#zły

#file_path = "/content/drive/My Drive/car-detection/labels/labels_val_txt/b1c66a42-6f7d68ca.txt"
file_path = "/content/drive/My Drive/car-detection/labels/labels_val_txt/fe1f55fa-19ba3600.txt"
with open(file_path, 'r') as f:
    content = f.read()

print(content)

{"name": "fe1f55fa-19ba3600.jpg", "attributes": {"weather": "snowy", "timeofday": "daytime", "scene": "city street"}, "timestamp": 10000, "labels": [{"id": "186022", "attributes": {"occluded": false, "truncated": false, "trafficLightColor": "R"}, "category": "traffic light", "box2d": {"x1": 391.794769, "y1": 378.069475, "x2": 401.776802, "y2": 390.153653202097}}, {"id": "186023", "attributes": {"occluded": false, "truncated": false, "trafficLightColor": "NA"}, "category": "car", "box2d": {"x1": 370.2999407103649, "y1": 408.015571, "x2": 398.033541, "y2": 431.722899}}, {"id": "186024", "attributes": {"occluded": false, "truncated": false, "trafficLightColor": "NA"}, "category": "traffic sign", "box2d": {"x1": 113.545617, "y1": 334.398083, "x2": 136.005191, "y2": 350.618886}}, {"id": "186025", "attributes": {"occluded": true, "truncated": false, "trafficLightColor": "NA"}, "category": "truck", "box2d": {"x1": 427.59205749483976, "y1": 376.821721, "x2": 449.191454, "y2": 425.8903201072645

In [ ]:
file_path = "/content/drive/My Drive/car-detection/labels/labels_val_txt/b5e0b513-da3b4402.txt"
# Odczytaj zawartość pliku

with open(file_path, 'r') as f:
    lines = f.readlines()
    print(lines[-2])
    if lines[-2].strip().endswith("},"):
        print("siema")
    else:
        print("niesiema")
# WAŻNA UWAGA - POMIJANA JEST PUSTA LINIJKA

    },

siema


In [ ]:
import os

def fix_json_files(folder_path):
    file_names = os.listdir(folder_path)

    for file_name in file_names:
        file_path = os.path.join(folder_path, file_name)

        # Odczytaj zawartość pliku
        with open(file_path, 'r') as f:
            lines = f.readlines()

        # Sprawdź, czy przedostatnia linia kończy się na "},"
        if lines[-2].strip().endswith("},"):
            lines = lines[:-2]  # Usuń linie po przedostatniej linii

        # Zapisz zmienioną zawartość do pliku
        with open(file_path, 'w') as f:
            f.writelines(lines)

        print(f"Sformatowano plik: {file_name}")

# Wprowadź ścieżkę do folderu zawierającego pliki tekstowe
folder_path = "/content/drive/My Drive/car-detection/labels/labels_val_txt"

# Wywołaj funkcję do naprawy plików
fix_json_files(folder_path)

Sformatowano plik: fe194677-e2d2ac8c.txt
Sformatowano plik: b5e0b513-da3b4402.txt
Sformatowano plik: b5f14913-d70897d9.txt
Sformatowano plik: b5eaada1-27688041.txt
Sformatowano plik: b5f9cb1e-c84e2ce9.txt
Sformatowano plik: b5ece4bd-17087c46.txt
Sformatowano plik: b5f6f399-ca14804c.txt
Sformatowano plik: b5e3d0f7-4ed391cd.txt
Sformatowano plik: b5fa32f1-4e9c7d32.txt
Sformatowano plik: b5ecfd7e-fcdaceb0.txt
Sformatowano plik: b5f7a789-10124031.txt
Sformatowano plik: b5fe17dd-b6648364.txt
Sformatowano plik: b5f7a789-6a9ed4ef.txt
Sformatowano plik: b5ff07b5-3411352a.txt
Sformatowano plik: b6036b55-6bc2eeef.txt
Sformatowano plik: b5fac784-638336fd.txt
Sformatowano plik: b5f7a789-c02a194e.txt
Sformatowano plik: b5ff07b5-b983e1a5.txt
Sformatowano plik: b61212a0-ca782534.txt
Sformatowano plik: b5fc0f1e-882a7cce.txt
Sformatowano plik: b603f785-b94d2adc.txt
Sformatowano plik: b603f785-de5db004.txt
Sformatowano plik: b5ff07b5-c8fe49f4.txt
Sformatowano plik: b5fc1373-94aaab40.txt
Sformatowano pli

In [ ]:
import os

def split_text_file(input_file):
    with open(input_file, 'r') as f:
        data = f.read()

    # Podziel dane na poszczególne obiekty na podstawie frazy "name":
    objects = data.split('"name": ')[1:]

    # Określ lokalizację folderu do zapisywania plików
    output_folder = "/content/drive/My Drive/car-detection/labels/labels_val_txt"
    os.makedirs(output_folder, exist_ok=True)

    for idx, obj in enumerate(objects):
        # Dodaj "name": z powrotem do obiektu, aby zachować poprawny format JSON
        obj = '{"name": ' + obj

        # Utwórz nazwę pliku na podstawie wartości "name"
        name_start_idx = obj.find('"') + 1
        name_end_idx = obj.find('.jpg"') + 4
        name = obj[name_start_idx:name_end_idx]
        txt_filename = name.replace(".jpg", ".txt")

        # Usuń frazę "name": z początku nazwy pliku
        txt_filename = txt_filename.replace('name": ', '')

        output_path = os.path.join(output_folder, txt_filename)

        # Zapisz obiekt JSON jako osobny plik tekstowy
        with open(output_path, 'w') as txt_file:
            txt_file.write(obj)

        print(f"Zapisano plik: {txt_filename}")

# Wprowadź ścieżkę do swojego pliku tekstowego
input_file = "/content/drive/My Drive/car-detection/labels/det_val_txt.txt"

# Uruchom funkcję do podziału pliku
split_text_file(input_file)

KeyboardInterrupt: ignored

In [ ]:
# RENAME FILES

import os

def format_file_names(folder_path):
    # Upewnij się, że ścieżka do folderu jest prawidłowa
    if not os.path.isdir(folder_path):
        print(f"Podana ścieżka '{folder_path}' nie jest folderem.")
        return

    # Przejdź przez wszystkie pliki w folderze
    for filename in os.listdir(folder_path):
        # Sprawdź, czy plik jest plikiem tekstowym
        if filename.endswith(".txt"):
            old_file_path = os.path.join(folder_path, filename)

            # Sformatuj nową nazwę pliku
            new_filename = filename.replace('name": "', '')
            new_file_path = os.path.join(folder_path, new_filename)

            # Zmień nazwę pliku
            os.rename(old_file_path, new_file_path)
            print(f"Zmieniono nazwę pliku: {filename} -> {new_filename}")

# Wprowadź ścieżkę do folderu zawierającego pliki
folder_path = "/content/drive/My Drive/car-detection/labels/labels_val_txt"

# Uruchom funkcję do formatowania nazw plików
format_file_names(folder_path)

In [ ]:
import json
import os

file_path = "/content/drive/My Drive/car-detection/labels/det_val.json"
output_folder = "/content/drive/My Drive/car-detection/labels/labels_val_txt"

json_file = "/content/drive/My Drive/car-detection/labels/det_val.json"

def create_txt_files(json_file, output_folder):
    with open(json_file, 'r') as f:
        data = json.load(f)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for item in data:
        if 'name' in item and 'regions' in item:
            file_name = item['name'].replace('.jpg', '.txt')
            file_path = os.path.join(output_folder, file_name)
            with open(file_path, 'w') as txt_file:
                for region in item['regions']:
                    for line in region.get('lines', []):
                        txt_file.write(line + '\n')
    create_txt_files(file_path, output_folder)

In [ ]:
from PIL import Image
import os

def check_resolution(image_folder):
    resolutions = set()

    for filename in os.listdir(image_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            filepath = os.path.join(image_folder, filename)
            with Image.open(filepath) as img:
                resolutions.add(img.size)

    if len(resolutions) == 1:
        print("Wszystkie obrazy mają tę samą rozdzielczość:", resolutions.pop())
    else:
        print("Obrazy mają różne rozdzielczości:", resolutions)

# Ścieżka do folderu z obrazami
image_folder_path = '/content/drive/My Drive/car-detection/images/val'
check_resolution(image_folder_path)

Wszystkie obrazy mają tę samą rozdzielczość: (1280, 720)


In [ ]:
from PIL import Image

try:
    #image = Image.open('/content/drive/My Drive/car-detection/images/val/b1d0a191-5490450b.jpg')
    image = Image.open('/content/drive/My Drive/car-detection/images/train/train100k/00225f53-67614580.jpg')

    image.show()
except Exception as e:
    print(f"Error opening the image: {e}")

In [ ]:
print(len(train_dataset))
print(len(train_loader))

In [ ]:
for l in train_loader:
  print(len(l))

In [ ]:
os.path.exists('/content/drive/My Drive/car-detection/images/train/train100k/8ec22bdd-58cd1abf.jpg')

In [ ]:
import os

def remove_files(folder):
    file_list = os.listdir(folder)
    for file_name in file_list:
        file_path = os.path.join(folder, file_name)
        os.remove(file_path)
        print(f"Usunięto plik: {file_name}")

# Wprowadź ścieżkę do folderu
folder = '/content/drive/My Drive/car-detection/labels/labels_val_txt'

# Wywołaj funkcję do usuwania plików
remove_files(folder)

In [ ]:
lista_sciezek = set([element["name"] for element in json_data])

In [ ]:
import json

file_path = "/content/drive/My Drive/car-detection/labels/det_train.json"
#file_path = "/content/drive/My Drive/car-detection/labels/det_val.json"

with open(file_path, "r") as file:
    json_data = json.load(file)

line_count = len(json_data)
print(f"Liczba linii w pliku JSON: {line_count}")

Liczba linii w pliku JSON: 69863


In [ ]:
print(json_data[0])

{'name': '0000f77c-6257be58.jpg', 'attributes': {'weather': 'clear', 'timeofday': 'daytime', 'scene': 'city street'}, 'timestamp': 10000, 'labels': [{'id': '0', 'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': 'G'}, 'category': 'traffic light', 'box2d': {'x1': 1125.902264, 'y1': 133.184488, 'x2': 1156.978645, 'y2': 210.875445}}, {'id': '1', 'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': 'G'}, 'category': 'traffic light', 'box2d': {'x1': 1156.978645, 'y1': 136.637417, 'x2': 1191.50796, 'y2': 210.875443}}, {'id': '2', 'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': 'NA'}, 'category': 'traffic sign', 'box2d': {'x1': 1105.66915985699, 'y1': 211.122087, 'x2': 1170.79037, 'y2': 233.566141}}, {'id': '3', 'attributes': {'occluded': False, 'truncated': True, 'trafficLightColor': 'NA'}, 'category': 'traffic sign', 'box2d': {'x1': 0.0, 'y1': 0.246631, 'x2': 100.381647, 'y2': 122.825696}}, {'id': '4', 'attributes': {'

In [ ]:
# PRINT LABELS

import os
import json

image_dir = "/content/drive/My Drive/car-detection/images/train/"
labels_file = "/content/drive/My Drive/car-detection/labels/bdd100k_labels_images_train.json"  # Ścieżka do pliku z etykietami (dostosuj do swojej struktury danych)

# Wczytanie danych etykiet
with open(labels_file, "r") as f:
    labels = json.load(f)

# Sprawdzenie informacji o nazwach plików w etykietach
for item in labels:
    file_name = item["name"]
    # Tutaj możesz wydrukować nazwy plików lub wykonać inne operacje na danych etykiet

# Porównanie nazw plików obrazów z informacjami w etykietach
file_list = os.listdir(image_dir)
for file_name in file_list:
    if file_name not in [item["name"] for item in labels]:
        print("Brak informacji w etykietach dla pliku:", file_name)

In [ ]:
# PRINT FILES

import os

image_dir = "/content/drive/My Drive/car-detection/images/train/"

# Wyświetlenie listy plików w folderze
file_list = os.listdir(image_dir)
for file_name in file_list:
    print(file_name)

In [ ]:
# COUNT FILES

!ls '/content/drive/My Drive/car-detection/labels/labels_val_txt' | wc -l

10000


In [ ]:
# MOVE FILES

import glob
import shutil

# Ścieżka do folderu źródłowego
src_folder = '/content/drive/My Drive/car-detection/images/train/train_4'

# Ścieżka do folderu docelowego
dst_folder = '/content/drive/My Drive/car-detection/images/train/train100k'

file_list = glob.glob(os.path.join(src_folder, '*.jpg'))
batch_size = 100  # Liczba plików w jednej partycji
num_batches = len(file_list) // batch_size + 1

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = start_idx + batch_size
    batch_files = file_list[start_idx:end_idx]

    for file_path in batch_files:
        file_name = os.path.basename(file_path)
        dst_path = os.path.join(dst_folder, file_name)
        shutil.move(file_path, dst_path)
        print(f"Przeniesiono plik: {file_name}")

In [ ]:
# DELETE FILES

import subprocess

# Polecenie ls
ls_output = subprocess.check_output("ls '/content/drive/My Drive/car-detection/images/train/train100k'", shell=True)

# Dekodowanie i przetworzenie wyniku polecenia ls
existing_files = ls_output.decode("utf-8").strip().split("\n")

# Twoja lista ścieżek
# lista_sciezek = [...]  # Twoja lista ścieżek

counter = 0

# Usuwanie plików .png, których nazwy nie znajdują się na liście
for file_name in existing_files:
    if file_name.endswith(".jpg") and file_name not in lista_sciezek:
        file_path = f"/content/drive/My Drive/car-detection/images/train/train100k/{file_name}"
        counter +=1
        subprocess.run(f"rm '{file_path}'", shell=True)

print(counter)

# Dalsze prace

## TODO

1.
Przygotowanie danych do treningu
Format json -> yaml

2.
Przejście przez cały tutorial
https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data
https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#train-on-custom-data

3.
Plan eksperymentów:


4.
Ewaluacja pretrenowanego modelu własnych danych BDD100K dataset (próba wyznaczenia prędkości)
Ramka, wyrzucenie szumów (skoki prawo, lewo) regresja liniowa

5.
Wykorzystanie pre-trenowanego modelu na bazie coco128.yaml
Dotrenowanie modelu na ograniczonej ilości klas zgodnie z labelami w coco128.yaml

6.
Wytrenowanie modelu od 0 na BDD100K DATASET (krok opcjonalny)


## TODO_2

1. DOCS postęp pracy

2. Na czym był trenowany yolov5s.pt i ten w tutorialu?


Mam obrazek o pewnej rozdzielczości i prostokąt opisany współrzędnymi. Chcę narysować ramkę, czego użyć?

Mam pre-trenowany model, który rozpoznaje różne klasy obiektów. Zwraca ramkę, ma klasę samochód, mogę zacząc eksperymenty z wyznaczaniem prędkości, a jednocześnie dotrenowywać model z moimi danymi